In [2]:
from dcicutils import ff_utils
from functions.notebook_functions import *

# Some attahcments on data are pointing to webdev s3 buckets, couldn't update them , so dowloaded and uploaded them again.
# the files and refernce excel can be found under Koray/wrangling (local files)



# get key from file
my_key = get_key('koray_data')

# all usable env names
all_envs = ['data', 'staging', 'fourfront-webdev', 'fourfront-mastertest', 'fourfront-hotseat']
# get admin key from s3
#my_key = ff_utils.get_authentication_with_server({}, ff_env = 'data')

In [4]:
# find all schemas with attachment
# list of all scheams with an existing url field
profiles = ff_utils.get_metadata('/profiles/', key=my_key)
urls = []

for a_p in profiles:
    props = profiles[a_p]['properties']
    for att_field in ['attachment', 'file_format_specification', 'cross_correlation_plot']:
        if att_field in props.keys():
            urls.append(a_p+'.'+att_field)

for i in sorted(urls):
    print(i)

Document.attachment
ExperimentSeq.attachment
FileCalibration.attachment
FileFormat.file_format_specification
FileMicroscopy.attachment
FileSetMicroscopeQc.attachment
Image.attachment
Protocol.attachment
Publication.attachment
PublicationTracking.attachment
QualityMetricFlag.attachment
SummaryStatistic.attachment
SummaryStatisticHiC.attachment


In [24]:
# Download link for old files are not working, checking all files to see if they are on S3 or not
from dcicutils import s3Utils
import os

my_env = 'data'
my_s3_util = s3Utils(env=my_env)
my_bucket = 'elasticbeanstalk-fourfront-webprod-blobs'

problematic = []

for i in urls:
    item , field = i.split('.')
    q= '/search/?type={it}&{fi}.href!=No value'.format(it=item, fi=field)
    print(q)
    all_res = ff_utils.search_metadata(q, key=my_key)
    if len(all_res)==0:
        print('None')
        continue
    
    for res in all_res:
        
        blob_id = res[field].get('blob_id')
        if not blob_id:
            print(res['display_title'], 'no blob id')
            continue
        
        # check if files are on s3
        head_info = my_s3_util.does_key_exist(blob_id, my_bucket)
        if not head_info:
            print(i['accession'], "does not have a file in S3")
            print()
        
        # check the download link
        href = res[field].get('href')
        if not href:
            print(res['display_title'], 'no href')
            continue
        href_at = res['uuid'] + '/' + href
        
        try:
            e = ff_utils.get_metadata(href_at, key=my_key)
            resp = str(e)
        except Exception as ex:
            resp = str(ex)

        if 'elasticbeanstalk-fourfront-webprod-blobs' in resp:
            print('fine')
            pass
        elif 'elasticbeanstalk-fourfront-webdev-blobs' in resp:
            problematic.append(res['uuid'])
            print('att in webdev')
        else:
            print(res['uuid'], 'other case', resp)
            print()    
    print(len(all_res))  
    print()

/search/?type=Document&attachment.href!=No value
fine
1

/search/?type=FileFormat&file_format_specification.href!=No value
None
/search/?type=SummaryStatistic&attachment.href!=No value
None
/search/?type=FileCalibration&attachment.href!=No value
fine
fine
att in webdev
att in webdev
att in webdev
att in webdev
att in webdev
att in webdev
att in webdev
att in webdev
10

/search/?type=SummaryStatisticHiC&attachment.href!=No value
None
/search/?type=Protocol&attachment.href!=No value
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine
fine

In [26]:
print(len(problematic))
print(problematic)

47
['0373f813-c86b-4d4c-96e7-6cf78aa7812e', 'cbc6e630-8677-4fdf-a70a-a33fb17c78ae', 'a5a40edb-8a36-4689-af9a-4d1aa1b0bd8d', 'c48258bc-5949-485e-aeea-e6fa5cc30aa3', '66a4dfc2-05bf-4d38-99f8-23ebff0dd7c3', '250b5413-fbea-4c33-9692-ad68bcfa771a', '37b870eb-0f63-4f0f-bb9b-34315f154160', 'fe1181a4-2c20-48a9-8350-93ce8317f6f2', 'fdc20e37-f22e-42dc-8026-0a132739bad1', '5347d0ae-198d-40ab-aac2-4ec6ee306e91', '093bd4e3-b799-46fd-b7a0-b3baf27089aa', '06248bc3-40ad-4f0a-9348-835e4a33d543', '5276c981-bf3f-431f-af49-b830326e2585', 'd2c7df31-7b34-47e6-9318-7bc081b7d18d', '77a6a279-23aa-48bf-871c-596a10a52951', '77612aad-e700-4098-84cf-0bc5b903486a', '55e6aa46-8019-4033-b427-f581b878daa1', '3c4ba318-bd1a-403a-a537-a55363898dbd', '2d2e7c78-bdad-4e18-a3d0-b8627b22c573', '3fadd793-a56f-4b6d-820a-82255a0ca580', '4b2f1d6a-35e6-49d4-8535-ed4e250a6135', '52749d55-018e-42a0-9e25-44cac9cac12c', '10f1ad74-81c3-479e-9765-7db7038f060c', 'ac7a45e6-59ab-47db-a8b4-4145d6a53f52', 'c764d799-4e15-4950-8f39-a73bdfbe9c8

In [10]:
import time
counter = 0
for pro in problematic:
    counter += 1
    status = ''
    status = [i for i in all_fq if i['uuid'] == pro][0]['status']
    ff_utils.patch_metadata({'status':'uploading'}, obj_id = pro, key = my_key)
    ff_utils.patch_metadata({'status':status}, obj_id = pro, key = my_key)
    if counter % 100 == 0:
        print(counter)
    

100
200


In [64]:
# dowload all attachments
from dcicutils import s3Utils
import os
import boto3
import botocore

BUCKET_NAME = 'elasticbeanstalk-fourfront-webprod-blobs'


s3_c = boto3.client("s3")

s3 = boto3.resource('s3')

my_folder = 'att_files'
my_env = 'data'
my_s3_util = s3Utils(env=my_env)
my_key = get_key('koray_data')

f_names = []
excel = []
for ui in problematic:
    print(ui)
    f_name = ''
    resp = ff_utils.get_metadata(ui, key = my_key)
    typ = resp['@type'][0]
    att = resp['attachment']
    blob = att['blob_id']
#     all_objects = s3_c.list_objects(Bucket = BUCKET_NAME, Prefix=blob) 
#     print(all_objects)
    print(blob)
    f_name = att['download']
    if f_name in f_names:
        print('duplicate')
        break
    f_names.append(f_name)
#     try:
#         s3.Bucket(BUCKET_NAME).download_file(blob, f_name)
#     except Exception as e:
#         print('download did not work')
#         print(e)
    excel.append([typ, resp['uuid'], f_name, blob])


0373f813-c86b-4d4c-96e7-6cf78aa7812e
5be33730-7128-435f-8f84-ba28c72264d1
cbc6e630-8677-4fdf-a70a-a33fb17c78ae
3fc10008-3b5d-4bd5-b446-c5cbeace83a7
a5a40edb-8a36-4689-af9a-4d1aa1b0bd8d
504145ec-9b48-43b1-8b65-8636ad4eca8c
c48258bc-5949-485e-aeea-e6fa5cc30aa3
a4dc9d4d-618d-46a2-aa9e-dba4f9edceb6
66a4dfc2-05bf-4d38-99f8-23ebff0dd7c3
e46ad1f3-a21e-49ef-ae03-5db8a6f63c84
250b5413-fbea-4c33-9692-ad68bcfa771a
daf304bd-7f10-4fe0-bbc9-67cca525e299
37b870eb-0f63-4f0f-bb9b-34315f154160
df470f4f-e589-434a-93bf-f5275fdb7d83
fe1181a4-2c20-48a9-8350-93ce8317f6f2
d9a0a049-44d8-46f8-8cca-be855ec8a2d3
fdc20e37-f22e-42dc-8026-0a132739bad1
1fc8c4ea-46ed-4a9e-a2ad-1dd7236ad944
5347d0ae-198d-40ab-aac2-4ec6ee306e91
d5b1442b-62f0-46c0-87ae-d624c5c675ff
093bd4e3-b799-46fd-b7a0-b3baf27089aa
33547a54-1529-4f98-bc3b-1e56e341a099
06248bc3-40ad-4f0a-9348-835e4a33d543
153a6675-1455-48d4-b328-ecb2c4897777
5276c981-bf3f-431f-af49-b830326e2585
27a1b7ef-3118-4768-a8a8-16ec8f3c9001
d2c7df31-7b34-47e6-9318-7bc081b7d18d
7

In [65]:
for i in excel:
    print ('\t'.join(i))

FileCalibration	0373f813-c86b-4d4c-96e7-6cf78aa7812e	README.txt	5be33730-7128-435f-8f84-ba28c72264d1
FileCalibration	cbc6e630-8677-4fdf-a70a-a33fb17c78ae	Ritland_2_doc.txt	3fc10008-3b5d-4bd5-b446-c5cbeace83a7
FileCalibration	a5a40edb-8a36-4689-af9a-4d1aa1b0bd8d	Pegoraro_doc.txt	504145ec-9b48-43b1-8b65-8636ad4eca8c
FileCalibration	c48258bc-5949-485e-aeea-e6fa5cc30aa3	Zipfel_doc.pdf	a4dc9d4d-618d-46a2-aa9e-dba4f9edceb6
FileCalibration	66a4dfc2-05bf-4d38-99f8-23ebff0dd7c3	Singer_doc.txt	e46ad1f3-a21e-49ef-ae03-5db8a6f63c84
FileCalibration	250b5413-fbea-4c33-9692-ad68bcfa771a	Ritland_doc.doc	daf304bd-7f10-4fe0-bbc9-67cca525e299
FileCalibration	37b870eb-0f63-4f0f-bb9b-34315f154160	Ries_doc.pdf	df470f4f-e589-434a-93bf-f5275fdb7d83
FileCalibration	fe1181a4-2c20-48a9-8350-93ce8317f6f2	Bewersdorf_doc.txt	d9a0a049-44d8-46f8-8cca-be855ec8a2d3
Protocol	fdc20e37-f22e-42dc-8026-0a132739bad1	4DN_in_situ_Hi-C_protocol_20170729.pdf	1fc8c4ea-46ed-4a9e-a2ad-1dd7236ad944
Protocol	5347d0ae-198d-40ab-aac2-4